# Étape 4 : Jointures (Merge) pour enrichir les ventes

## Préparation à la jointure

 ### 1.- Chargement des 3 fichiers

In [25]:
order_lines = pd.read_csv('C:\\Documents\\Abdarare\\FDS-FRST-SDMIA\\Python\\Mini_Projet_Pandas\\Livrables\\Données_nettoyées\\order_lines_clean.csv')
customers = pd.read_csv('C:\\Documents\\Abdarare\\FDS-FRST-SDMIA\\Python\\Mini_Projet_Pandas\\Livrables\\Données_nettoyées\\customers_clean.csv')
products = pd.read_csv('C:\\Documents\\Abdarare\\FDS-FRST-SDMIA\\Python\\Mini_Projet_Pandas\\Livrables\\Données_nettoyées\\products_clean.csv')

### 2.- Vérification des clés de jointure

In [26]:
# Clé : customer_id
customer_ids_orders = order_lines['customer_id'].nunique()
customer_ids_customers = customers['customer_id'].nunique()

missing_customers = order_lines[~order_lines['customer_id'].isin(customers['customer_id'])]

# Clé : product_id
product_ids_orders = order_lines['product_id'].nunique()
product_ids_products = products['product_id'].nunique()

missing_products = order_lines[~order_lines['product_id'].isin(products['product_id'])]

### 3.- Jointure

In [27]:
# Jointure 1 : ventes-clients sur customer_id
print(f"   Lignes dans order_lines : {len(order_lines)}")

orders_with_customers = pd.merge(
    order_lines,              
    customers,               
    on='customer_id',        
    how='left',               
    suffixes=('', '_cust')    
)

print(f"   Lignes dans orders_with_customers : {len(orders_with_customers)}")

# Jointure 2 : résultat-produits sur product_id
print(f"   Lignes dans orders_with_customers : {len(orders_with_customers)}")

orders_enriched = pd.merge(
    orders_with_customers,
    products,                 
    on='product_id',          
    how='left',               
    suffixes=('', '_prod')    
)

print(f"   Lignes dans orders_enriched : {len(orders_enriched)}")

   Lignes dans order_lines : 2188
   Lignes dans orders_with_customers : 2188
   Lignes dans orders_with_customers : 2188
   Lignes dans orders_enriched : 2188


## Contrôle de la jointure et recalcul

### 1.- Contrôle de la qualité de la jointure

In [34]:
print(f"   Lignes initiales (order_lines)     : {len(order_lines)}")
print(f"   Lignes après merge 1 (+ customers) : {len(orders_with_customers)}")
print(f"   Lignes après merge 2 (+ products)  : {len(orders_enriched)}")

colonnes_dupliquees = [col for col in orders_enriched.columns if col.endswith('_cust') or col.endswith('_prod')]

if colonnes_dupliquees:
    print(f"\n Colonnes dupliquées détectées : {colonnes_dupliquees}")
    print("   Raison : ces colonnes existent dans order_lines ET dans customers/products")
    print("   Solution : utiliser les colonnes avec suffixe ou supprimer les doublons")

# Colonnes importantes à vérifier
colonnes_a_verifier = ['age', 'gender', 'city_cust', 'segment', 'unit_price', 'brand', 'category']
colonnes_presentes = [col for col in colonnes_a_verifier if col in orders_enriched.columns]

for col in colonnes_presentes:
    nb_nan = orders_enriched[col].isna().sum()
    pct_nan = (nb_nan / len(orders_enriched)) * 100
    if nb_nan > 0:
        print(f"    {col:<15} : {nb_nan} NaN ({pct_nan:.1f}%)")
    else:
        print(f"    {col:<15} : aucune valeur manquante")

   Lignes initiales (order_lines)     : 2188
   Lignes après merge 1 (+ customers) : 2188
   Lignes après merge 2 (+ products)  : 2188

 Colonnes dupliquées détectées : ['city_cust', 'segment_cust', 'category_prod', 'unit_price_prod']
   Raison : ces colonnes existent dans order_lines ET dans customers/products
   Solution : utiliser les colonnes avec suffixe ou supprimer les doublons
    age             : aucune valeur manquante
    gender          : aucune valeur manquante
    city_cust       : aucune valeur manquante
    segment         : aucune valeur manquante
    unit_price      : aucune valeur manquante
    brand           : aucune valeur manquante
    category        : aucune valeur manquante


### 2.- Recalcul des colonnes business après enrichissement

In [31]:
if 'unit_price' in orders_enriched.columns:
    
    orders_enriched['gross_amount_calc'] = orders_enriched['unit_price'] * orders_enriched['quantity']
    
    if orders_enriched['discount_pct'].dtype == 'object':
        orders_enriched['discount_pct'] = pd.to_numeric(
            orders_enriched['discount_pct'].astype(str).str.replace('%', ''),
            errors='coerce'
        )
        if orders_enriched['discount_pct'].max() > 1:
            orders_enriched['discount_pct'] = orders_enriched['discount_pct'] / 100
    
    orders_enriched['net_amount_calc'] = orders_enriched['gross_amount_calc'] * (1 - orders_enriched['discount_pct'])
    
    orders_enriched['amount_diff'] = abs(orders_enriched['net_amount'] - orders_enriched['net_amount_calc'])
    
    lignes_suspectes = orders_enriched[orders_enriched['amount_diff'] > 0.01]
    nb_suspectes = len(lignes_suspectes)
    pct_suspectes = (nb_suspectes / len(orders_enriched)) * 100

## Mini-analyse 

In [32]:
segment_col = 'segment'
category_col = 'category'

if 'segment_cust' in orders_enriched.columns:
    segment_col = 'segment_cust'
if 'category_prod' in orders_enriched.columns:
    category_col = 'category_prod'

print(f"\nUtilisation des colonnes : {segment_col} × {category_col}")

pivot_segment_category = orders_enriched.pivot_table(
    values='net_amount',         
    index=segment_col,           
    columns=category_col,        
    aggfunc='sum',               
    fill_value=0,                
    margins=True,               
    margins_name='TOTAL'        
)

pivot_segment_category = pivot_segment_category.sort_values('TOTAL', ascending=False)

print(pivot_segment_category.round(2).to_string())


Utilisation des colonnes : segment_cust × category_prod
category_prod  Accessoire      Cloud      Cours      Laptop     Livre   Logiciel       TOTAL
segment_cust                                                                                
TOTAL            50526.71  154973.15  203189.21  1410706.64  40702.46  163560.27  2023658.44
Étudiant         16269.23   49248.88   77883.94   564245.09  13493.37   52649.34   773789.85
Professionnel    18279.27   56899.34   69387.16   507813.50  15235.28   58983.62   726598.17
Indépendant       8542.39   23821.75   35781.16   193132.45   7845.77   31424.59   300548.11
Entreprise        7435.82   25003.18   20136.95   145515.60   4128.04   20502.72   222722.31


In [33]:
orders_enriched.to_csv(r'C:\Documents\Abdarare\FDS-FRST-SDMIA\Python\Mini_Projet_Pandas\Livrables\Jointure_et_pivot table\orders_enriched.csv', index=False)
pivot_segment_category.to_csv('C:\\Documents\\Abdarare\\FDS-FRST-SDMIA\\Python\\Mini_Projet_Pandas\\Livrables\\Jointure_et_pivot table\\pivot_segment_category.csv', index=False)

# Commentaires

##### Cette jointure confirme la parfaite intégrité des données, avec un maintien rigoureux de 2 188 lignes tout au long des processus de jointure, ce qui prouve l'absence de doublons ou de pertes d'informations. L'analyse de qualité ne révèle aucune valeur manquante sur les dimensions essentielles (âge, genre, segment, catégorie), validant ainsi la fiabilité des clés de jointure entre les fichiers clients, produits et ventes. Bien que des collisions de colonnes aient été détectées et gérées par des suffixes (ex: city_cust), les contrôles de cohérence business sur les montants nets ont permis d'isoler les écarts de calcul potentiels. Enfin, la synthèse via le tableau croisé dynamique met en lumière la domination du segment "Étudiant" et de la catégorie "Laptop", qui constituent les principaux moteurs du chiffre d'affaires total de 2 023 658,44.